In [1]:
import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(tensor([128002, 128002, 128000,   5269,    527,    499]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<|reserved_special_token_0|><|reserved_special_token_0|><|begin_of_text|>how are you')

In [2]:
import json

with open('dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=16)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(16,
 {'prompt': 'Human: context= CREATE TABLE table_name_49 (redline__rpm_ INTEGER, engine_code VARCHAR, year VARCHAR) question= What is the average Redline RPM of engines with engine code M57TUD30, made after 2002? Assistant:',
  'chosen': 'SELECT AVG(redline__rpm_) FROM table_name_49 WHERE engine_code = "m57tud30" AND year > 2002',
  'rejected': '',
  'response': 'SELECT AVG(redline__rpm_) FROM table_name_49 WHERE engine_code = "m57tud30" AND year > 2002'})

In [3]:
%run 1.model.ipynb

model_actor = torch.load('model/rlhf').to('cuda')

In [4]:
from util import get_generate

for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=128)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = get_generate(model_actor,
                            input_ids,
                            tokenizer.eos_token_id,
                            tokenizer.pad_token_id,
                            max_length=256)

    generate = generate[0, 128:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select avg(INTROPHY__WITH_) from TABLE_NAME_49 where ENGINE_CODE = "COLD COLEETHEATI" and YEAR > 30<|end_of_text|>
SELECT AVG(redline__rpm_) FROM table_name_49 WHERE engine_code = "m57tud30" AND year > 2002
select sum(REBOUNDS) from TABLE_NAME_21 where NAME = "CORK & MIRI" and RANK > 8<|end_of_text|>
SELECT SUM(rebounds) FROM table_name_21 WHERE name = "fedor likholitov" AND rank > 8
select avg(ELECTION) from TABLE_NAME_30 where MUNICIPALITY = "MILLIPPER"<|end_of_text|>
SELECT AVG(election) FROM table_name_30 WHERE municipality = "monza"
select FILM_TITLE_USED_IN_NOMINATION from TABLE_NAME_18 where COUNTRY = "SWITZILIPPINES"<|end_of_text|>
SELECT film_title_used_in_nomination FROM table_name_18 WHERE country = "netherlands"
select SEASON from TABLE_24575253_4 where DIVISION_THREE = "dUNA eAST sT"<|end_of_text|>
SELECT season FROM table_24575253_4 WHERE division_four = "Wallington New Foresters"
select MARGIN_OF_VICTORY from TABLE_NAME_34 where RUNNER_S__UP = "GREG NORMAN"<|end_of_text|